## Don't run this notebook

In [ ]:
replace_names = []
count = 0
for i in G1.nodes:
    if i not in all_data["characters_infobox"]:
        replace_names.append(i)
        count += 1
print(replace_names)

In [ ]:
import asyncio
import aiohttp
from bs4 import BeautifulSoup

async def fetch_header(session, name):
    url = "https://naruto.fandom.com/wiki/{}".format(name)
    
    try:
        async with session.get(url) as response:
            if response.status == 200:
                html = await response.text()
                soup = BeautifulSoup(html, 'html.parser')
                # Extract the content within the specified span element
                span_element = soup.find('span', class_='mw-page-title-main')
                header = span_element.text.strip() if span_element else None
                
                return {'original_name': name, 'header': header}
            else:
                print("URL: {}, Status: {}".format(url, response.status))
    except aiohttp.ClientError as e:
        print("Error for URL {}: {}".format(url, e))
    
    return {'original_name': name, 'header': None}  # Return a dictionary even when header is not found or there is an error

async def main():
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_header(session, name) for name in replace_names]
        results = await asyncio.gather(*tasks)
    
    return results

result_records = await main()

# Now you can work with the 'result_records' variable as needed
# For example, you can iterate through it and print each record:
for result in result_records:
    print(f"{result['original_name']}: {result['header']}")


In [ ]:
corrected_node_names = result_records
corrected_node_names = {index: value for index, value in enumerate(corrected_node_names)}
corrected_node_names.pop(21)
corrected_node_names.pop(29)

In [ ]:
with open('../data/corrected_node_names.json', 'w') as json_file:
    json.dump(corrected_node_names, json_file)

In [ ]:
import json

# Specify the path to your JSON file
json_file_path_episodes_seasons_characters = '/Users/lukasjonsson/Desktop/DTU/Kandidat/3. semester/Social graphs & interactions/Project/socialGraphsProject/data/episodes_seasons_characters.json'

# Open the file and load its contents as a dictionary
with open(json_file_path_episodes_seasons_characters, 'r') as file:
    episodes_seasons_characters = json.load(file)

In [ ]:
import json

# Assuming corrected_node_names is a dictionary with 'original_name' and 'header' keys

# Iterate over the data and update names
for key, episodes in episodes_seasons_characters.items():
    for episode in episodes:
        characters = episode.get('Characters', [])
        for i in range(len(characters)):
            for node_id, correction in corrected_node_names.items():
                if correction['original_name'] == characters[i]:
                    characters[i] = correction['header']

# Save the updated data back to the JSON file
with open('../data/episodes_seasons_characters_updated.json', 'w') as json_file:
    json.dump(episodes_seasons_characters, json_file)
